### xfeat GitHub  
https://github.com/pfnet-research/xfeat

### xfeat tutorial  
https://github.com/pfnet-research/xfeat/blob/master/examples/xfeat_tutorial_notebook.ipynb  

### kaggle titanic
https://www.kaggle.com/c/titanic/data

### Kaggleで使えるFeather形式を利用した特徴量管理法
https://amalog.hateblo.jp/entry/kaggle-feature-management  

### 【Python】バイナリファイルへの書き込み（モードwb）【入門第61回】
https://naruport.com/blog/2019/9/20/python-tutorial-write-to-binary-file/

### 【データサイエンティストへの道⑴】lightGBMの軽ーい実装とまとめ
https://note.com/___n_h__n__/n/na69a79719d56

In [1]:
import pandas as pd
import numpy as np
import os

from IPython.display import display

import xfeat
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, ArithmeticCombinations,  LambdaEncoder

import pickle

from sklearn.model_selection import train_test_split

# lightGBM
import lightgbm as lgb

# 二乗平均平方根誤差 (RMSE)
from sklearn.metrics import mean_squared_error

from sklearn import metrics

In [2]:
path = os.getcwd() + "/"
d_name = ["train", "test"]

In [3]:
for i in d_name:
    xfeat.utils.compress_df(pd.read_csv(path + i + ".csv")).to_feather(path + i + ".ftr")

In [4]:
# Check the serialized data.
dtrain = pd.read_feather("./train.ftr")
dtest = pd.read_feather("./test.ftr")

display(dtrain.head(3))
display(dtrain.tail(3))
display(dtrain.dtypes)
display(dtest.head(3))
display(dtest.tail(3))
display(dtest.dtypes)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250000,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283302,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925000,None,S


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.450001,None,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.000000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.750000,None,Q


PassengerId      int16
Survived          int8
Pclass            int8
Name            object
Sex             object
Age            float32
SibSp             int8
Parch             int8
Ticket          object
Fare           float32
Cabin           object
Embarked        object
dtype: object

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.250000,None,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.050000,None,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.358299,None,C


PassengerId      int16
Pclass            int8
Name            object
Sex             object
Age            float32
SibSp             int8
Parch             int8
Ticket          object
Fare           float32
Cabin           object
Embarked        object
dtype: object

In [5]:
# カテゴリデータ（object）のみ集計　SelectCategorical().fit_transform()
display(SelectCategorical().fit_transform(dtrain).head())
display(SelectCategorical().fit_transform(dtest).head())

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


,Name,Sex,Ticket,Cabin,Embarked
0,"Kelly, Mr. James",male,330911,None,Q
1,"Wilkes, Mrs. James (Ellen Needs)",female,363272,None,S
2,"Myles, Mr. Thomas Francis",male,240276,None,Q
3,"Wirz, Mr. Albert",male,315154,None,S
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,3101298,None,S


In [6]:
gs = pd.read_csv(path + "gender_submission.csv")
display(gs.head())
display(gs.tail())
display(len(gs))

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0


418

In [7]:
# ラベルエンコーディング

# Takes categorical columns from the data frame and performs label encoding on them.
# The converted data is stored in the column with suffix defined in `output_suffix`.
# By defining `output_suffix=""`, it is possible to store the result in the same column.

# データフレームからカテゴリ型の列を取り出し、ラベルエンコーディングを行います。
# 変換されたデータは `output_suffix` で定義されたサフィックスを持つ列に格納されます。
# output_suffix=""`を定義することで、同じカラムに結果を格納することができます。

encoder1 = Pipeline([
    # 除外するカラムをexclude_colsに指定
    SelectCategorical(exclude_cols=["Name"]),
    LabelEncoder(output_suffix=""),
])

In [8]:
display(encoder1.fit_transform(dtrain).head())
display(encoder1.fit_transform(dtest).head())

,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,0,1
2,1,2,-1,0
3,1,3,1,0
4,0,4,-1,0


,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,-1,1
2,0,2,-1,0
3,0,3,-1,1
4,1,4,-1,1


In [9]:
encoder2 = Pipeline([
    # 除外するカラムをexclude_colsに指定
    SelectCategorical(exclude_cols=["Name"]),

    # If there are many categorical columns,
    # users can specify the columns to be combined with `input_cols` kwargs.
    # `r=2` specifies the number of columns to combine the columns.
    
    # カテゴリ列が多数存在する場合.
    # ユーザーは `input_cols` kwargs で結合するカラムを指定することができます。
    # `r=2` は結合するカラムの数を指定します。
    
    ConcatCombination(drop_origin=True, output_suffix="", r=2),
    
    LabelEncoder(output_suffix=""),
])

### 4つの項目から2つを選ぶ組み合わせ　4C2 = 6　6通りが出力される

In [10]:
display(encoder2.fit_transform(dtrain).head())
display(encoder2.fit_transform(dtest).head())

,SexTicket,SexCabin,SexEmbarked,TicketCabin,TicketEmbarked,CabinEmbarked
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,2,2,2,2,2,0
3,3,3,2,3,3,2
4,4,0,0,4,4,0


,SexTicket,SexCabin,SexEmbarked,TicketCabin,TicketEmbarked,CabinEmbarked
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,2,0,0,2,2,0
3,3,0,2,3,3,1
4,4,1,1,4,4,1


In [11]:
# SelectNumerical extracts only the column of numerical data from the input dataframe.
# 入力データフレームから数値データの列のみを抽出します

# exclude_cols で出力しないカラムを指定

encoder3_1 = Pipeline([
    # 除外するカラムをexclude_colsに指定
    SelectNumerical(exclude_cols=["PassengerId", "Survived"]),
    LabelEncoder(output_suffix=""),
])

display(encoder3_1.fit_transform(dtrain).head())

,Pclass,Age,SibSp,Parch,Fare
0,0,0,0,0,0
1,1,1,0,0,1
2,0,2,1,0,2
3,1,3,0,0,3
4,0,3,1,0,4


In [12]:
encoder3_2 = Pipeline([
    # 除外するカラムをexclude_colsに指定
    SelectNumerical(exclude_cols=["PassengerId"]),
    LabelEncoder(output_suffix=""),
])

# test.csvは Survived がないのでencoder3_1を適用するとエラーが発生する
display(encoder3_2.fit_transform(dtest).head())

,Pclass,Age,SibSp,Parch,Fare
0,0,0,0,0,0
1,0,1,1,0,1
2,1,2,0,0,2
3,0,3,0,0,3
4,0,4,1,1,4


In [13]:
# xfeat.ArithmeticCombinations　creates new columns by applying arithmetic combinations.
# 算術的な組み合わせを適用して新しい列を作成します。

encoder4_1 = Pipeline([
    SelectNumerical(exclude_cols=["PassengerId", "Survived"]),
    ArithmeticCombinations(
        drop_origin=True,
        operator="+",
        r=2,
        output_suffix="",
    ),
])

display(encoder4_1.fit_transform(dtrain).head())

,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,25.0,4,3,10.250000,23.0,22.0,29.250000,1,8.250000,7.250000
1,39.0,2,1,72.283302,39.0,38.0,109.283302,1,72.283302,71.283302
2,29.0,3,3,10.925000,26.0,26.0,33.924999,0,7.925000,7.925000
3,36.0,2,1,54.099998,36.0,35.0,88.099998,1,54.099998,53.099998
4,38.0,3,3,11.050000,35.0,35.0,43.049999,0,8.050000,8.050000


In [14]:
encoder4_2 = Pipeline([
    SelectNumerical(exclude_cols=["PassengerId"]),
    ArithmeticCombinations(
        drop_origin=True,
        operator="+",
        r=2,
        output_suffix="",
    ),
])

display(encoder4_2.fit_transform(dtest).head())

,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,37.5,3,3,10.8292,34.5,34.5,42.329201,0,7.8292,7.8292
1,50.0,4,3,10.0000,48.0,47.0,54.000000,1,8.0000,7.0000
2,64.0,2,2,11.6875,62.0,62.0,71.687500,0,9.6875,9.6875
3,30.0,3,3,11.6625,27.0,27.0,35.662498,0,8.6625,8.6625
4,25.0,4,4,15.2875,23.0,23.0,34.287498,2,13.2875,13.2875


In [15]:
# xfeat.LambdaEncoder takes a lambda function as an argument and transforms the columns of the data frame.
# ラムダ関数を引数に取り、データフレームの列を変換します。

encoder5 = Pipeline([
    SelectNumerical(exclude_cols=["PassengerId", "Survived"]),
    ArithmeticCombinations(
        drop_origin=True,
        operator="+",
        r=2,
        output_suffix="",
    ),

    # 
    LambdaEncoder(
        lambda x: float(str(x)[:5]),
        output_prefix="",
        output_suffix="",
        drop_origin=True,
    ),
])

encoder5.fit_transform(dtrain).head()

,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,25.0,4.0,3.0,10.25,23.0,22.0,29.25,1.0,8.250,7.250
1,39.0,2.0,1.0,72.28,39.0,38.0,109.20,1.0,72.280,71.280
2,29.0,3.0,3.0,10.92,26.0,26.0,33.92,0.0,7.925,7.925
3,36.0,2.0,1.0,54.09,36.0,35.0,88.09,1.0,54.090,53.090
4,38.0,3.0,3.0,11.05,35.0,35.0,43.04,0.0,8.050,8.050


In [16]:
encoder6 = Pipeline([
    SelectNumerical(exclude_cols=["PassengerId"]),
    ArithmeticCombinations(
        drop_origin=True,
        operator="+",
        r=2,
        output_suffix="",
    ),

    LambdaEncoder(
        # 
        lambda x: float(str(x)[:5]),
        output_prefix="",
        output_suffix="",
        drop_origin=True,
    ),
])

encoder6.fit_transform(dtest).head()

,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,37.5,3.0,3.0,10.82,34.5,34.5,42.32,0.0,7.829,7.829
1,50.0,4.0,3.0,10.00,48.0,47.0,54.00,1.0,8.000,7.000
2,64.0,2.0,2.0,11.68,62.0,62.0,71.68,0.0,9.687,9.687
3,30.0,3.0,3.0,11.66,27.0,27.0,35.66,0.0,8.662,8.662
4,25.0,4.0,4.0,15.28,23.0,23.0,34.28,2.0,13.280,13.280


### 【用語解説】シリアライズ・デシリアライズとは
http://cloudcafe.tech/?p=2639

シリアライズ（serialize）とは、プログラミングでオプジェクト化されたデータを、ファイルやストレージに保存したり、ネットワークで送受信したりできるような形に変換することを言います。  

逆に、シリアライズされたデータをプログラミングで扱えるようにオブジェクトの型に復元することをデシリアライズ（deserialize）といいます。  

In [17]:
# Serialize/Deserialize
# The parameters of the encoder can be serialized/deserialized by pickle.
# エンコーダのパラメータをPickleでシリアライズ/デシリアライズすることができます。

encoder7 = Pipeline([
    SelectCategorical(exclude_cols=["Name"]),
    LabelEncoder(output_suffix=""),
])

dtrain_encoded = encoder7.fit_transform(dtrain)

# 書き込み
with open("label_train.pkl", "wb") as f:
    pickle.dump(encoder7, f)
    
dtrain_encoded.head()

,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,0,1
2,1,2,-1,0
3,1,3,1,0
4,0,4,-1,0


In [18]:
# 読込み
with open("label_train.pkl", "rb") as f:
    encoder7 = pickle.load(f)

encoder7.transform(dtrain).head()

,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,0,1
2,1,2,-1,0
3,1,3,1,0
4,0,4,-1,0


In [19]:
# dtrain のエンコーダーでdtestを読み込むとおかしな感じに。4行目のTicketが405に・・・
encoder7.transform(dtest).head()

,Sex,Ticket,Cabin,Embarked
0,0,-1,-1,2
1,1,-1,-1,0
2,0,-1,-1,2
3,0,-1,-1,0
4,1,405,-1,0


In [20]:
encoder8 = Pipeline([
    SelectCategorical(exclude_cols=["Name"]),
    LabelEncoder(output_suffix=""),
])

dtest_encoded2 = encoder8.fit_transform(dtest)

# 書き込み
with open("label_test.pkl", "wb") as f:
    pickle.dump(encoder8, f)
    
dtest_encoded2.head()

,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,-1,1
2,0,2,-1,0
3,0,3,-1,1
4,1,4,-1,1


In [21]:
# 読込み
with open("label_test.pkl", "rb") as f:
    encoder8 = pickle.load(f)

encoder8.transform(dtest).head()

,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,-1,1
2,0,2,-1,0
3,0,3,-1,1
4,1,4,-1,1


The Label encoding mapping is kept in train.csv and test.csv. Unseen values are assigned to -1 in this case.  

ラベルエンコーディングマッピングは train.csv と test.csv に保持されます。この場合、未見の値は-1に代入されます。  

In [22]:
# SelectCategorical
dtrain_en1 = encoder1.fit_transform(dtrain)
dtest_en1 = encoder1.fit_transform(dtest)

# SelectCategorical + ConcatCombination
dtrain_en2 = encoder2.fit_transform(dtrain)
dtest_en2 = encoder2.fit_transform(dtest)

# SelectNumerical
dtrain_en3_1 = encoder3_1.fit_transform(dtrain)
dtest_en3_2 = encoder3_2.fit_transform(dtest)

# SelectNumerical + ArithmeticCombinations
dtrain_en4_1 = encoder4_1.fit_transform(dtrain)
dtest_en4_2 = encoder4_2.fit_transform(dtest)

In [23]:
display(dtrain_en1.head())
display(dtest_en1.head())

display(dtrain_en2.head())
display(dtest_en2.head())

display(dtrain_en3_1.head())
display(dtest_en3_2.head())

display(dtrain_en4_1.head())
display(dtest_en4_2.head())

,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,0,1
2,1,2,-1,0
3,1,3,1,0
4,0,4,-1,0


,Sex,Ticket,Cabin,Embarked
0,0,0,-1,0
1,1,1,-1,1
2,0,2,-1,0
3,0,3,-1,1
4,1,4,-1,1


,SexTicket,SexCabin,SexEmbarked,TicketCabin,TicketEmbarked,CabinEmbarked
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,2,2,2,2,2,0
3,3,3,2,3,3,2
4,4,0,0,4,4,0


,SexTicket,SexCabin,SexEmbarked,TicketCabin,TicketEmbarked,CabinEmbarked
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,2,0,0,2,2,0
3,3,0,2,3,3,1
4,4,1,1,4,4,1


,Pclass,Age,SibSp,Parch,Fare
0,0,0,0,0,0
1,1,1,0,0,1
2,0,2,1,0,2
3,1,3,0,0,3
4,0,3,1,0,4


,Pclass,Age,SibSp,Parch,Fare
0,0,0,0,0,0
1,0,1,1,0,1
2,1,2,0,0,2
3,0,3,0,0,3
4,0,4,1,1,4


,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,25.0,4,3,10.250000,23.0,22.0,29.250000,1,8.250000,7.250000
1,39.0,2,1,72.283302,39.0,38.0,109.283302,1,72.283302,71.283302
2,29.0,3,3,10.925000,26.0,26.0,33.924999,0,7.925000,7.925000
3,36.0,2,1,54.099998,36.0,35.0,88.099998,1,54.099998,53.099998
4,38.0,3,3,11.050000,35.0,35.0,43.049999,0,8.050000,8.050000


,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,37.5,3,3,10.8292,34.5,34.5,42.329201,0,7.8292,7.8292
1,50.0,4,3,10.0000,48.0,47.0,54.000000,1,8.0000,7.0000
2,64.0,2,2,11.6875,62.0,62.0,71.687500,0,9.6875,9.6875
3,30.0,3,3,11.6625,27.0,27.0,35.662498,0,8.6625,8.6625
4,25.0,4,4,15.2875,23.0,23.0,34.287498,2,13.2875,13.2875


### データ結合

In [24]:
train = pd.concat([dtrain_en1, dtrain_en2], axis=1)
train = pd.concat([train, dtrain_en3_1], axis=1)
train = pd.concat([train, dtrain_en4_1], axis=1)
train = pd.concat([train, dtrain["Survived"]], axis=1)

test = pd.concat([dtest_en1, dtest_en2], axis=1)
test = pd.concat([test, dtest_en3_2], axis=1)
test = pd.concat([test, dtest_en4_2], axis=1)

In [25]:
display(train.head())
display(train.shape)

display(test.head())
display(test.shape)

,Sex,Ticket,Cabin,Embarked,SexTicket,SexCabin,SexEmbarked,TicketCabin,TicketEmbarked,CabinEmbarked,...,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare,Survived
0,0,0,-1,0,0,0,0,0,0,0,...,4,3,10.250000,23.0,22.0,29.250000,1,8.250000,7.250000,0
1,1,1,0,1,1,1,1,1,1,1,...,2,1,72.283302,39.0,38.0,109.283302,1,72.283302,71.283302,1
2,1,2,-1,0,2,2,2,2,2,0,...,3,3,10.925000,26.0,26.0,33.924999,0,7.925000,7.925000,1
3,1,3,1,0,3,3,2,3,3,2,...,2,1,54.099998,36.0,35.0,88.099998,1,54.099998,53.099998,1
4,0,4,-1,0,4,0,0,4,4,0,...,3,3,11.050000,35.0,35.0,43.049999,0,8.050000,8.050000,0


(891, 26)

,Sex,Ticket,Cabin,Embarked,SexTicket,SexCabin,SexEmbarked,TicketCabin,TicketEmbarked,CabinEmbarked,...,PclassAge,PclassSibSp,PclassParch,PclassFare,AgeSibSp,AgeParch,AgeFare,SibSpParch,SibSpFare,ParchFare
0,0,0,-1,0,0,0,0,0,0,0,...,37.5,3,3,10.8292,34.5,34.5,42.329201,0,7.8292,7.8292
1,1,1,-1,1,1,1,1,1,1,1,...,50.0,4,3,10.0000,48.0,47.0,54.000000,1,8.0000,7.0000
2,0,2,-1,0,2,0,0,2,2,0,...,64.0,2,2,11.6875,62.0,62.0,71.687500,0,9.6875,9.6875
3,0,3,-1,1,3,0,2,3,3,1,...,30.0,3,3,11.6625,27.0,27.0,35.662498,0,8.6625,8.6625
4,1,4,-1,1,4,1,1,4,4,1,...,25.0,4,4,15.2875,23.0,23.0,34.287498,2,13.2875,13.2875


(418, 25)

### lightGBM

In [26]:
features = [
    "Sex",
    "Ticket",
    "Cabin",
    "Embarked",
    "SexTicket",
    "SexCabin",
    "SexEmbarked",
    "TicketCabin",
    "TicketEmbarked",
    "CabinEmbarked",
    "PclassSibSp",
    "PclassParch",
    "PclassFare",
    "AgeSibSp",
    "AgeParch",
    "AgeFare",
    "SibSpParch",
    "SibSpFare",
    "ParchFare"
]

In [27]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'binary', # 二項分類
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1,
    'bagging_fraction': 0.75,
    'bagging_freq': 10, 
    'colsample_bytree': 0.75
}

In [28]:
X_train, X_test, y_train, y_test = train_test_split(train[features], train["Survived"], random_state=42)

In [29]:
#LGB用のデータに変形
lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_test, y_test)

In [30]:
model_lgb = lgb.train(params, lgb_train, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [lgb_train, lgb_val], verbose_eval = 1) 

[1]	training's rmse: 0.464012	valid_1's rmse: 0.470618
Training until validation scores don't improve for 50 rounds
[2]	training's rmse: 0.444789	valid_1's rmse: 0.451615
[3]	training's rmse: 0.430337	valid_1's rmse: 0.439494
[4]	training's rmse: 0.415922	valid_1's rmse: 0.426907
[5]	training's rmse: 0.403638	valid_1's rmse: 0.416687
[6]	training's rmse: 0.393206	valid_1's rmse: 0.408827
[7]	training's rmse: 0.38484	valid_1's rmse: 0.402414
[8]	training's rmse: 0.377441	valid_1's rmse: 0.396552
[9]	training's rmse: 0.370421	valid_1's rmse: 0.391288
[10]	training's rmse: 0.36469	valid_1's rmse: 0.388461
[11]	training's rmse: 0.357085	valid_1's rmse: 0.383922
[12]	training's rmse: 0.350798	valid_1's rmse: 0.380484
[13]	training's rmse: 0.345583	valid_1's rmse: 0.37822
[14]	training's rmse: 0.34146	valid_1's rmse: 0.376896
[15]	training's rmse: 0.3377	valid_1's rmse: 0.375954
[16]	training's rmse: 0.334339	valid_1's rmse: 0.37414
[17]	training's rmse: 0.330876	valid_1's rmse: 0.37335
[18]

In [31]:
val_pred = model_lgb.predict(X_test)
val_score = np.sqrt(mean_squared_error(val_pred, y_test))
print(f'rmse score は {val_score}')

rmse score は 0.36439652043851445


In [32]:
pred = model_lgb.predict(test[features])
len(pred)

418

In [33]:
gs.shape

(418, 2)

In [34]:
gs["Survived"] = pred

In [35]:
display(gs.head())
display(gs.tail())

,PassengerId,Survived
0,892,0.088763
1,893,0.469688
2,894,0.095865
3,895,0.286390
4,896,0.562130


,PassengerId,Survived
413,1305,0.064279
414,1306,0.952058
415,1307,0.083875
416,1308,0.064279
417,1309,0.420913


In [36]:
# 0.6以上の場合は 1、0.6未満の場合は 0 に置換
gs["Survived"].mask(gs["Survived"] >= 0.6, 1, inplace=True)
gs["Survived"].mask(gs["Survived"] < 0.6, 0, inplace=True)

In [37]:
display(gs.head())
display(gs.tail())

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0


,PassengerId,Survived
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0
417,1309,0.0


In [38]:
# csvへの出力
gs.to_csv(path + "submission.csv", index=False)